In [ ]:
import pandas as pd
from google.colab import files
import io

uploaded = files.upload()

Saving leafly_scraped_all.csv to leafly_scraped_all.csv


In [ ]:
'''
This is the first stage of the cleaning process. Data from the Otreeba API that was previously
cleaned is merged with the data that I scraped from leafly. I have scraped leafly again to 
get more of the reported feelings, THC content, ratings, and images. 

In the second stage, the dataframe created from this merge will be merged with a dataframe
created during our labs project. That previous dataframe is data from Kaggle merged with an
API that keeps that Kaggle dataframe up to date. 
'''

In [ ]:
api_merge = pd.read_csv(io.BytesIO(uploaded['jose_strains_cleaned.csv']))

In [ ]:
leafly = pd.read_csv(io.BytesIO(uploaded['leafly_scraped_all.csv']))

## Cleaning api_merge

In [ ]:
api_merge.head(10)

,Unnamed: 0,name,ocpc,qr,url,image,seedCompany,genetics,lineage,children,createdAt,updatedAt
0,0,#1K,9XVU7PZUEC000000000000000,https://www.cannabisreports.com/strain-reports...,https://www.cannabisreports.com/strain-reports...,NaN,"{'name': 'Unknown Breeder', 'ocpc': '9XVU70000...","{'names': False, 'ocpc': False}",[],"['PHE7M3VGAU000000000000000', 'PHE7MVCT7X00000...",2015-06-16 22:10:20,2015-06-16 22:10:20
1,1,#1K PC,PHE7MVCT7X000000000000000,https://www.cannabisreports.com/strain-reports...,https://www.cannabisreports.com/strain-reports...,NaN,"{'name': 'Purple Caper Seeds', 'ocpc': 'PHE7M0...","{'names': '#1K x Purple Caper', 'ocpc': '9XVU7...","{'Brazil': 'BR', 'India': 'IN'}",False,2015-06-16 22:11:13,2015-06-16 22:11:13
2,2,#32,NRJKFUN7AM000000000000000,https://www.cannabisreports.com/strain-reports...,https://www.cannabisreports.com/strain-reports...,NaN,"{'name': 'Archive Seed Bank', 'ocpc': 'NRJKF00...","{'names': 'Albert Walker x Manic', 'ocpc': '9X...",[],['NRJKF4797R000000000000000'],2015-03-25 23:19:03,2015-03-25 23:19:03
3,3,#32 x Face-Off OG Kush,NRJKF4797R000000000000000,https://www.cannabisreports.com/strain-reports...,https://www.cannabisreports.com/strain-reports...,NaN,"{'name': 'Archive Seed Bank', 'ocpc': 'NRJKF00...","{'names': '#32 x Faceoff Cut OG Kush', 'ocpc':...",{'Afghanistan': 'AF'},['NRJKFZ6R2N000000000000000'],2015-03-25 23:21:56,2015-03-25 23:21:56
4,4,'78 LA OG Affie,9XVU7N3NWM000000000000000,https://www.cannabisreports.com/strain-reports...,https://www.cannabisreports.com/strain-reports...,NaN,"{'name': 'Aficionado Seeds', 'ocpc': 'KWZ9M000...",{'names': 'Lemon Thai x Colombian Gold x Pakis...,"{'Colombia': 'CO', 'Pakistan': 'PK', 'Thailand...",False,2015-08-07 20:38:43,2015-08-07 20:39:03
5,5,'86 Black UW,9XVU7ZFZZC000000000000000,https://www.cannabisreports.com/strain-reports...,https://www.cannabisreports.com/strain-reports...,NaN,"{'name': 'Unknown Breeder', 'ocpc': '9XVU70000...","{'names': 'U Dub (UW)', 'ocpc': '9XVU7MTPCH000...",[],['U9XW7GKGAG000000000000000'],2015-06-17 16:20:43,2015-06-17 16:20:43
6,6,(Ducksfoot x Lowryder) x AK-47,9XVU7U6Y9A000000000000000,https://www.cannabisreports.com/strain-reports...,https://www.cannabisreports.com/strain-reports...,https://www.cannabisreports.com/images/strains...,"{'name': 'Unknown Breeder', 'ocpc': '9XVU70000...","{'names': '( Ducksfoot x Lowryder ) x AK-47', ...","{'Afghanistan': 'AF', 'Colombia': 'CO', 'Mexic...",False,2014-07-25 02:16:51,2014-07-25 02:19:06
7,7,(Early Pearl/Skunk No. 1/Northern Lights #5) x...,2NGAWYEM6F000000000000000,https://www.cannabisreports.com/strain-reports...,https://www.cannabisreports.com/strain-reports...,NaN,"{'name': 'Seed Bank', 'ocpc': '2NGAW0000000000...",{'names': '(Early Pearl / Skunk No. 1 / Northe...,"{'Afghanistan': 'AF', 'Colombia': 'CO', 'India...",False,2008-08-15 19:20:00,2014-06-07 08:14:28
8,8,(G-13 x Blue Widow) x Killer Queen,UVLGYGJP92000000000000000,https://www.cannabisreports.com/strain-reports...,https://www.cannabisreports.com/strain-reports...,NaN,"{'name': 'NCGA', 'ocpc': 'UVLGY000000000000000...","{'names': ""(Airborne's G13 x Blue Widow) x Kil...","{'Afghanistan': 'AF', 'Brazil': 'BR', 'Colombi...",False,2008-08-15 19:20:00,2014-06-07 08:21:48
9,9,(Hash Plant / Northern Lights #1 / ?) x Swaziland,2NGAWFLA9R000000000000000,https://www.cannabisreports.com/strain-reports...,https://www.cannabisreports.com/strain-reports...,NaN,"{'name': 'Seed Bank', 'ocpc': '2NGAW0000000000...",{'names': '(Hash Plant (Sensi) / Northern Ligh...,"{'Afghanistan': 'AF', 'Swaziland': 'SZ'}",False,2008-08-15 19:20:00,2014-06-07 08:14:40


In [ ]:
leafly.tail()

,Unnamed: 0,name,description,terpene_description,technical_terpenes,parents,children,reported_feelings,pic,thc_content,rating
3405,3405,Zour Apples,Plantman Jack and Ethos Glue from Ethos Geneti...,[],[],[],[],"['Energetic', 'Happy', 'Relaxed', 'Creative', ...",https://public.leafly.com/strains/flowers/zour...,[],4.7 7 reviews
3406,3406,Zsweet Insanity,"Bred by Ethos Genetics, Zsweet Inzanity is a c...",[],[],"Rainmaker, Durban Poison",[],"['Euphoric', 'Giggly', 'Happy', 'Tingly', 'Upl...",https://public.leafly.com/strains/flowers/zswe...,[],4.4 8 reviews
3407,3407,Hai Fantasy,Thai Fantasy by Kannabia Seed Company is a sat...,[],[],[],[],[],https://public.leafly.com/strains/flowers/hai-...,[],1 1 reviews
3408,3408,Alenuihaha,ʻAlenuihāhā is a sativa-dominant hybrid by Pua...,[],[],"Kona Gold, Maui Wowie",[],"['Aroused', 'Creative', 'Energetic', 'Euphoric...",https://public.leafly.com/strains/flowers/alen...,[],5 7 reviews
3409,3409,88 G13 Hashplant,Created in 1988 by NDNguy and widely circulate...,[],[],"Hash Plant, G 13",[],"['Relaxed', 'Tingly']",https://public.leafly.com/strains/flowers/88-g...,[],5 2 reviews


In [ ]:
#clean strain names
#drop qr, ocpc, url, children, createdAt, updatedAt, seedCompany
#clean seedCompany, genetics, image
#see how many of the same strains are in leafly and api_merge

In [ ]:
def clean_names(test):
  test = test.replace('#', '')
  test = test.replace('+', '')
  test = test.replace('?', '')
  test = test.replace("'", '')
  test = test.replace('-', ' ')

  return test

#clean up the names so they have a chance to merge correctly 
api_merge['name'] = api_merge['name'].apply(clean_names)

In [ ]:
#drop unnecessary columns
api_merge = api_merge.drop(['Unnamed: 0', 'qr', 'ocpc', 'url', 'children', 'createdAt', 'updatedAt', 'seedCompany'], axis=1)

In [ ]:
#remove image links if they don't have a real picture
#I am not sure why this isn't working correctly, this error isn't something I should run into here
#ERROR! Session/line number was not unique in database. History logging moved to new session 60

#will return to this later

no_image = api_merge['image'][3]

for i in api_merge['image']:
  if i == 'https://www.cannabisreports.com/images/strains/no_image.png':
    i = i.replace('https://www.cannabisreports.com/images/strains/no_image.png', 'None')
  else:
    pass

In [ ]:
def clean_genes(i):
  if i == "{'names': False, 'ocpc': False}":
    i = i.replace("{'names': False, 'ocpc': False}", 'None')
  head, sep, tail = i.partition(',')
  i = head
  i = i.replace("{'names': ", '')
  i = i.replace("'", "")
  return i 

#remove everything from genetics except for prior strains 
api_merge['genetics'] = api_merge['genetics'].apply(clean_genes)

#clean the cleaned genetics with the name cleaner
api_merge['genetics'] = api_merge['genetics'].apply(clean_names)

In [ ]:
def clean_lineage(i):
  head, sep, tail = i.partition(',')
  i = head
  head, sep, tail = i.partition(':')
  i = head
  i = i.replace('{', '')
  i = i.replace("'", "")
  return i 

#get country of origin only 
api_merge['lineage'] = api_merge['lineage'].apply(clean_lineage)

In [ ]:
api_merge.head()
#lookin good

,name,image,genetics,lineage
0,1K,NaN,None,[]
1,1K PC,NaN,1K x Purple Caper,Brazil
2,32,NaN,Albert Walker x Manic,[]
3,32 x Face Off OG Kush,NaN,32 x Faceoff Cut OG Kush,Afghanistan
4,78 LA OG Affie,NaN,Lemon Thai x Colombian Gold x Pakistani Landrace,Colombia


### Clean new leafly data

In [ ]:
leafly.head(20)

,Unnamed: 0,name,description,terpene_description,technical_terpenes,parents,children,reported_feelings,pic,thc_content,rating
0,0,100 Og,$100 OG is a 50/50 hybrid strain that packs a ...,[],[],Og Kush,[],"['Happy', 'Energetic', 'Focused', 'Sleepy', 'T...",https://public.leafly.com/strains/flowers/100-...,This info is sourced from our readers and is n...,2.9 508 reviews
1,1,98 White Widow,The ‘98 Aloha White Widow is an especially pot...,[],[],[],Aloha Grape Stomper,"['Euphoric', 'Happy', 'Relaxed', 'Uplifted', '...",https://public.leafly.com/strains/flowers/98-w...,This info is sourced from our readers and is n...,4.8 15 reviews
2,2,1024,1024 is a sativa-dominant hybrid bred in Spain...,[],[],[],Aloha Grape Stomper,"['Creative', 'Focused', 'Euphoric', 'Happy', '...",https://public.leafly.com/strains/flowers/1024...,This info is sourced from our readers and is n...,4.5 55 reviews
3,3,10Th Planet,Ethos Genetics crossed Planet of the Grapes an...,[],[],Quattro Kush,[],"['Euphoric', 'Giggly', 'Happy', 'Hungry', 'Rel...",https://public.leafly.com/strains/flowers/10th...,This info is sourced from our readers and is n...,5 3 reviews
4,4,12 Year Og,The Bank Cannabis Genetics crossed some unknow...,[],[],Og Kush,[],[],https://public.leafly.com/strains/flowers/12-y...,This info is sourced from our readers and is n...,3 2 reviews
5,5,13 Dawgs,13 Dawgs is a hybrid of G13 and Chemdawg genet...,[],[],"G 13, Chemdawg",[],"['Euphoric', 'Sleepy', 'Creative', 'Hungry', '...",https://public.leafly.com/strains/flowers/13-d...,This info is sourced from our readers and is n...,4.1 12 reviews
6,6,2 Fast 2 Vast,The sequel to Fast and Vast by Heavyweight See...,[],[],"G 13, Chemdawg",[],[],https://public.leafly.com/strains/flowers/2-fa...,This info is sourced from our readers and is n...,Write a review
7,7,22,22 is a Jack Herer cross from Cali Connection....,"['Peppery', 'Citrus', 'Hoppy']","['width:170%', 'Caryophyllene', 'width:114%', ...","G 13, Chemdawg",[],"['Creative', 'Energetic', 'Euphoric', 'Focused...",https://public.leafly.com/strains/flowers/22-f...,18%,3.7 10 reviews
8,8,24K Gold,"Also known as Kosher Tangie, 24k Gold is a 60%...","['Herbal', 'Pine', 'Citrus']","['width:170%', 'Myrcene', 'width:114%', 'Pinen...","Kosher Kush, Tangie",[],"['Happy', 'Relaxed', 'Relaxed', 'Euphoric', 'R...",https://public.leafly.com/strains/flowers/24k-...,20.5%,4.6 260 reviews
9,9,3 Bears Og,3 Bears OG by Mephisto Genetics is an autoflow...,[],[],Triangle Kush,[],"['Relaxed', 'Creative', 'Euphoric', 'Giggly', ...",https://public.leafly.com/strains/flowers/3-be...,20.5%,4.5 8 reviews


In [ ]:
#drop extra column

#clean thc
# -- remove no thc filler text

#clean pics
# -- remove pngs
# they are all just the pngs :(

#clean ratings 
# -- make new column for how many reviews
# -- make rating column only the rating

#clean terpenes
# -- turn them into proper json format

In [ ]:
leafly = leafly.drop('Unnamed: 0', axis = 1)

In [ ]:
#remove thc filler text 
big = 'This info is sourced from our readers and is not a substitute for professional medical advice. Seek the advice of a health professional before using cannabis for a medical condition.'

leafly['thc_content'] = leafly['thc_content'].apply(lambda x: x.replace(big, ''))

In [ ]:
#remove pngs from pic

def clean_pics(i):
  if i[-4:-1] == '.pn':
    print('dont want this')
  else:
    print('do want this')

#leafly['pic'] = leafly['pic'].apply(clean_pics)

In [ ]:
#clean ratings 
# -- make new column for how many reviews
# -- make rating column only the rating
series = []
rating = []
for i in leafly['rating']:
  head, sep, tail = i.partition(' ')
  rating.append(head)
  series.append(tail)

leafly['rating'] = rating
leafly['reviews'] = series

In [ ]:
#clean any leftovers left in rating and reviews
#reviews -- remove 'a review'
#rating -- remove 'Write'

leafly['rating'] = leafly['rating'].replace('Write', 'None')
leafly['reviews'] = leafly['reviews'].replace('a review', 'None')
leafly['reviews'] = leafly['reviews'].replace('reviews', '')

In [ ]:
#okay so there are too many reported feelings to fit in our graph in a pretty way, I need to
#replace some words with a word that means the same thing

#euphoric <--- happy, uplifed, giggly
#aroused <--- tingly
#anxious <--- paranoid
#dry mouth <--- dry eyes
#energetic <--- talkative


In [ ]:
def clean_feels(i):
  i = i.replace(" 'Happy',", " 'Euphoric',")
  i = i.replace("['Happy',", "['Euphoric',")
  i = i.replace(" 'Happy']", " 'Euphoric']")
  i = i.replace(" 'Uplifted',", " 'Euphoric',")
  i = i.replace("['Uplifted',", "['Euphoric',")
  i = i.replace("'Uplifted']", " 'Euphoric']")
  i = i.replace(" 'Giggly',", " 'Euphoric',")
  i = i.replace("['Giggly',", "['Euphoric',")
  i = i.replace(" 'Giggly']", " 'Euphoric']")
  i = i.replace(" 'Tingly',", " 'Aroused',")
  i = i.replace("['Tingly',", "['Aroused',")
  i = i.replace(" 'Tingly']", " 'Aroused']")
  i = i.replace(" 'Paranoid',", " 'Anxious',")
  i = i.replace("['Paranoid',", "['Anxious',")
  i = i.replace(" 'Paranoid']", " 'Anxious']")
  i = i.replace(" 'Dry Eyes',", " 'Dry Mouth',")
  i = i.replace("['Dry Eyes',", "['Dry Mouth',")
  i = i.replace(" 'Dry Eyes']", " 'Dry Mouth']")
  i = i.replace(" 'Talkative',", " 'Energetic',")
  i = i.replace("['Talkative',", "['Energetic',")
  i = i.replace(" 'Talkative']", " 'Energetic']")
  return i

In [ ]:
leafly['reported_feelings'] = leafly['reported_feelings'].apply(clean_feels)

In [ ]:
leafly['reported_feelings'][2]

#after reveiwing many individual rows, I have confidence that this has finally worked :D

"['Creative', 'Focused', 'Euphoric', 'Happy', 'Uplifted', 'Creative', 'Euphoric', 'Relaxed', 'Uplifted', 'Creative', 'Energetic', 'Euphoric', 'Focused', 'Happy', 'Creative', 'Energetic', 'Focused', 'Happy', 'Hungry', 'Uplifted']"

In [ ]:
#clean terpenes
# -- turn them into proper json format

#this was copied from another notebook because it was last minute cleaning,
#so you may have to tweak it a little on the first run

def clean_terps(x):
  if "'width" in x:
    x = x.replace("'width", "{'terpine'")
  head, sep, tail = x.partition(']')
  x = head
  x+="}']"

  if ':' in x:
    x = x.replace(':', ": '" )

  if ', {' in x:
    x = x.replace(', {', "}', ")

  #if ' ter' in x:
  #  x = x.replace(' ter', " '{ter")

  if "', " in x:
    x = x.replace("%', '", "%', 'name': '")

  if " 'terpine'" in x:
    x = x.replace(" 'terpine'", " '{'terpine'")

  if "[{'terpine'" in x:
    x = x.replace("[{'terpine'", " ['{'terpine'")  

  return x

In [ ]:
#function cannot be applied without manuvering around the nans

def replace_nans(y):
  if pd.isnull(y):
    return 'placeholder'
  else:
    return y

leafly['technical_terpenes'] = leafly['technical_terpenes'].apply(replace_nans)
leafly.head()

,name,description,terpene_description,technical_terpenes,parents,children,reported_feelings,pic,thc_content,rating,reviews
0,100 Og,$100 OG is a 50/50 hybrid strain that packs a ...,[],[],Og Kush,[],"['Happy', 'Energetic', 'Focused', 'Sleepy', 'T...",https://public.leafly.com/strains/flowers/100-...,,2.9,508 reviews
1,98 White Widow,The ‘98 Aloha White Widow is an especially pot...,[],[],[],Aloha Grape Stomper,"['Euphoric', 'Happy', 'Relaxed', 'Uplifted', '...",https://public.leafly.com/strains/flowers/98-w...,,4.8,15 reviews
2,1024,1024 is a sativa-dominant hybrid bred in Spain...,[],[],[],Aloha Grape Stomper,"['Creative', 'Focused', 'Euphoric', 'Happy', '...",https://public.leafly.com/strains/flowers/1024...,,4.5,55 reviews
3,10Th Planet,Ethos Genetics crossed Planet of the Grapes an...,[],[],Quattro Kush,[],"['Euphoric', 'Giggly', 'Happy', 'Hungry', 'Rel...",https://public.leafly.com/strains/flowers/10th...,,5,3 reviews
4,12 Year Og,The Bank Cannabis Genetics crossed some unknow...,[],[],Og Kush,[],[],https://public.leafly.com/strains/flowers/12-y...,,3,2 reviews


In [ ]:
#now we can use the function to fix terpenes

leafly['technical_terpenes'] = leafly['technical_terpenes'].apply(clean_terps)
leafly.head(20)

,name,description,terpene_description,technical_terpenes,parents,children,reported_feelings,pic,thc_content,rating,reviews
0,100 Og,$100 OG is a 50/50 hybrid strain that packs a ...,[],[}'],Og Kush,[],"['Happy', 'Energetic', 'Focused', 'Sleepy', 'T...",https://public.leafly.com/strains/flowers/100-...,,2.9,508 reviews
1,98 White Widow,The ‘98 Aloha White Widow is an especially pot...,[],[}'],[],Aloha Grape Stomper,"['Euphoric', 'Happy', 'Relaxed', 'Uplifted', '...",https://public.leafly.com/strains/flowers/98-w...,,4.8,15 reviews
2,1024,1024 is a sativa-dominant hybrid bred in Spain...,[],[}'],[],Aloha Grape Stomper,"['Creative', 'Focused', 'Euphoric', 'Happy', '...",https://public.leafly.com/strains/flowers/1024...,,4.5,55 reviews
3,10Th Planet,Ethos Genetics crossed Planet of the Grapes an...,[],[}'],Quattro Kush,[],"['Euphoric', 'Giggly', 'Happy', 'Hungry', 'Rel...",https://public.leafly.com/strains/flowers/10th...,,5,3 reviews
4,12 Year Og,The Bank Cannabis Genetics crossed some unknow...,[],[}'],Og Kush,[],[],https://public.leafly.com/strains/flowers/12-y...,,3,2 reviews
5,13 Dawgs,13 Dawgs is a hybrid of G13 and Chemdawg genet...,[],[}'],"G 13, Chemdawg",[],"['Euphoric', 'Sleepy', 'Creative', 'Hungry', '...",https://public.leafly.com/strains/flowers/13-d...,,4.1,12 reviews
6,2 Fast 2 Vast,The sequel to Fast and Vast by Heavyweight See...,[],[}'],"G 13, Chemdawg",[],[],https://public.leafly.com/strains/flowers/2-fa...,,None,None
7,22,22 is a Jack Herer cross from Cali Connection....,"['Peppery', 'Citrus', 'Hoppy']","['{'terpine': '170%', 'name': 'Caryophyllene'...","G 13, Chemdawg",[],"['Creative', 'Energetic', 'Euphoric', 'Focused...",https://public.leafly.com/strains/flowers/22-f...,18%,3.7,10 reviews
8,24K Gold,"Also known as Kosher Tangie, 24k Gold is a 60%...","['Herbal', 'Pine', 'Citrus']","['{'terpine': '170%', 'name': 'Myrcene'}', '{...","Kosher Kush, Tangie",[],"['Happy', 'Relaxed', 'Relaxed', 'Euphoric', 'R...",https://public.leafly.com/strains/flowers/24k-...,20.5%,4.6,260 reviews
9,3 Bears Og,3 Bears OG by Mephisto Genetics is an autoflow...,[],[}'],Triangle Kush,[],"['Relaxed', 'Creative', 'Euphoric', 'Giggly', ...",https://public.leafly.com/strains/flowers/3-be...,20.5%,4.5,8 reviews


In [ ]:
cols = list(leafly.columns)

In [ ]:
#remove the garbage created by the nan replacement

for i in cols:
  leafly[i] = leafly[i].replace(["[}']"], float('nan'))

## Time to merge

In [ ]:
leafly.head(20)

,name,description,terpene_description,technical_terpenes,parents,children,reported_feelings,pic,thc_content,rating,reviews
0,100 Og,$100 OG is a 50/50 hybrid strain that packs a ...,[],NaN,Og Kush,[],"['Happy', 'Energetic', 'Focused', 'Sleepy', 'T...",https://public.leafly.com/strains/flowers/100-...,,2.9,508 reviews
1,98 White Widow,The ‘98 Aloha White Widow is an especially pot...,[],NaN,[],Aloha Grape Stomper,"['Euphoric', 'Happy', 'Relaxed', 'Uplifted', '...",https://public.leafly.com/strains/flowers/98-w...,,4.8,15 reviews
2,1024,1024 is a sativa-dominant hybrid bred in Spain...,[],NaN,[],Aloha Grape Stomper,"['Creative', 'Focused', 'Euphoric', 'Happy', '...",https://public.leafly.com/strains/flowers/1024...,,4.5,55 reviews
3,10Th Planet,Ethos Genetics crossed Planet of the Grapes an...,[],NaN,Quattro Kush,[],"['Euphoric', 'Giggly', 'Happy', 'Hungry', 'Rel...",https://public.leafly.com/strains/flowers/10th...,,5,3 reviews
4,12 Year Og,The Bank Cannabis Genetics crossed some unknow...,[],NaN,Og Kush,[],[],https://public.leafly.com/strains/flowers/12-y...,,3,2 reviews
5,13 Dawgs,13 Dawgs is a hybrid of G13 and Chemdawg genet...,[],NaN,"G 13, Chemdawg",[],"['Euphoric', 'Sleepy', 'Creative', 'Hungry', '...",https://public.leafly.com/strains/flowers/13-d...,,4.1,12 reviews
6,2 Fast 2 Vast,The sequel to Fast and Vast by Heavyweight See...,[],NaN,"G 13, Chemdawg",[],[],https://public.leafly.com/strains/flowers/2-fa...,,None,None
7,22,22 is a Jack Herer cross from Cali Connection....,"['Peppery', 'Citrus', 'Hoppy']","['{'terpine': '170%', 'name': 'Caryophyllene'...","G 13, Chemdawg",[],"['Creative', 'Energetic', 'Euphoric', 'Focused...",https://public.leafly.com/strains/flowers/22-f...,18%,3.7,10 reviews
8,24K Gold,"Also known as Kosher Tangie, 24k Gold is a 60%...","['Herbal', 'Pine', 'Citrus']","['{'terpine': '170%', 'name': 'Myrcene'}', '{...","Kosher Kush, Tangie",[],"['Happy', 'Relaxed', 'Relaxed', 'Euphoric', 'R...",https://public.leafly.com/strains/flowers/24k-...,20.5%,4.6,260 reviews
9,3 Bears Og,3 Bears OG by Mephisto Genetics is an autoflow...,[],NaN,Triangle Kush,[],"['Relaxed', 'Creative', 'Euphoric', 'Giggly', ...",https://public.leafly.com/strains/flowers/3-be...,20.5%,4.5,8 reviews


In [ ]:
print('Leafly shape:', leafly.shape, 'api_merge shape:', api_merge.shape)

Leafly shape: (3410, 11) api_merge shape: (8756, 4)


In [ ]:
counts_samename = 0
strains_in_both = []
for i in api_merge['name']:
  for x in leafly['name']:
    if x == i:
      counts_samename += 1
      strains_in_both += ([i])
counts_samename
#strains_in_both

1322

In [ ]:
leafly_merge = pd.merge(api_merge, leafly, how='outer', on='name')

In [ ]:
#get rid of duplicates
leafly_merge = leafly_merge.drop_duplicates(subset=['name'], keep='first')

In [ ]:
leafly_merge.shape

(10796, 14)

In [ ]:
leafly.head()

,name,description,terpene_description,technical_terpenes,parents,children,reported_feelings,pic,thc_content,rating,reviews
0,100 Og,$100 OG is a 50/50 hybrid strain that packs a ...,[],NaN,Og Kush,[],"['Happy', 'Energetic', 'Focused', 'Sleepy', 'T...",https://public.leafly.com/strains/flowers/100-...,,2.9,508 reviews
1,98 White Widow,The ‘98 Aloha White Widow is an especially pot...,[],NaN,[],Aloha Grape Stomper,"['Euphoric', 'Happy', 'Relaxed', 'Uplifted', '...",https://public.leafly.com/strains/flowers/98-w...,,4.8,15 reviews
2,1024,1024 is a sativa-dominant hybrid bred in Spain...,[],NaN,[],Aloha Grape Stomper,"['Creative', 'Focused', 'Euphoric', 'Happy', '...",https://public.leafly.com/strains/flowers/1024...,,4.5,55 reviews
3,10Th Planet,Ethos Genetics crossed Planet of the Grapes an...,[],NaN,Quattro Kush,[],"['Euphoric', 'Giggly', 'Happy', 'Hungry', 'Rel...",https://public.leafly.com/strains/flowers/10th...,,5,3 reviews
4,12 Year Og,The Bank Cannabis Genetics crossed some unknow...,[],NaN,Og Kush,[],[],https://public.leafly.com/strains/flowers/12-y...,,3,2 reviews


In [ ]:
leafly_merge.head(70)

,name,image,genetics,lineage,description,terpene_description,technical_terpenes,parents,children,reported_feelings,pic,thc_content,rating,reviews
0,1K,NaN,None,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1K PC,NaN,1K x Purple Caper,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,32,NaN,Albert Walker x Manic,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,32 x Face Off OG Kush,NaN,32 x Faceoff Cut OG Kush,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,78 LA OG Affie,NaN,Lemon Thai x Colombian Gold x Pakistani Landrace,Colombia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,8 K,NaN,None,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116,8 K x G13,NaN,8 K x G 13 (G13),[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,818 Headband,https://www.cannabisreports.com/images/strains...,Original Sour D x San Fernando Valley OG (SFV OG),Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118,818 Pure SFV OG S1,https://www.cannabisreports.com/images/strains...,San Fernando Valley OG (SFV OG),Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
leafly_merge.tail()

,name,image,genetics,lineage,description,terpene_description,technical_terpenes,parents,children,reported_feelings,pic,thc_content,rating,reviews
10848,Zour Apples,NaN,NaN,NaN,Plantman Jack and Ethos Glue from Ethos Geneti...,[],[],[],[],"['Energetic', 'Euphoric', 'Relaxed', 'Creative...",https://public.leafly.com/strains/flowers/zour...,[],4.7,7 reviews
10849,Zsweet Insanity,NaN,NaN,NaN,"Bred by Ethos Genetics, Zsweet Inzanity is a c...",[],[],"Rainmaker, Durban Poison",[],"['Euphoric', 'Euphoric' 'Euphoric', 'Aroused',...",https://public.leafly.com/strains/flowers/zswe...,[],4.4,8 reviews
10850,Hai Fantasy,NaN,NaN,NaN,Thai Fantasy by Kannabia Seed Company is a sat...,[],[],[],[],[],https://public.leafly.com/strains/flowers/hai-...,[],1,1 reviews
10851,Alenuihaha,NaN,NaN,NaN,ʻAlenuihāhā is a sativa-dominant hybrid by Pua...,[],[],"Kona Gold, Maui Wowie",[],"['Aroused', 'Creative', 'Energetic', 'Euphoric...",https://public.leafly.com/strains/flowers/alen...,[],5,7 reviews
10852,88 G13 Hashplant,NaN,NaN,NaN,Created in 1988 by NDNguy and widely circulate...,[],[],"Hash Plant, G 13",[],"['Relaxed', 'Aroused']",https://public.leafly.com/strains/flowers/88-g...,[],5,2 reviews


In [ ]:
leafly_merge.shape

(10796, 14)

In [ ]:
leafly_merge.to_csv(r'full_leafly_merge.csv')